In [1]:
import numpy as np
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, GlobalMaxPooling2D, MaxPooling2D, Conv2D, GlobalAveragePooling2D 
from numpy import array
from keras import optimizers
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
from keras.applications.resnet50 import ResNet50
import keras.applications
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from os.path import isfile, join
from os import listdir
import os, os.path
import shutil
from random import shuffle
from shutil import copyfile
%matplotlib inline
from keras.callbacks import ModelCheckpoint
import cv2
import sys
import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range=0.2,
        zoom_range=0.2,     
        rotation_range=15,
        width_shift_range=.15,
        height_shift_range=.15)
test_datagen = ImageDataGenerator(rescale=1./255)
train_path = "/home/t-judai/kaggle-link/datasets/grassknoted/asl-alphabet/asl_alphabet_train"
valid_path = "/home/t-judai/kaggle-link/datasets/grassknoted/asl-alphabet/asl_alphabet_valid"
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(224, 224),
        batch_size=24,
        class_mode='categorical',
        shuffle = True)

validation_generator = train_datagen.flow_from_directory(
        valid_path,
        target_size=(224, 224),
        batch_size=24,
        class_mode='categorical',
        shuffle = False)   

Found 60871 images belonging to 29 classes.
Found 26129 images belonging to 29 classes.


In [3]:
#base_model = keras.applications.resnet50.ResNet50(include_top=False, weights = "imagenet")
sys.path.append("/home/hongzl/code/utils/keras_models")
from resnet152_keras import resnet152_model
with tf.device('/device:GPU:0'):
    base_model = resnet152_model(weights_path="/work/resnet152_weights_tf.h5",classes=29)

x = base_model.output
#x = GlobalAveragePooling2D()(x) 
#x = Dense(29, activation="softmax")(x)
model = Model(base_model.input, x, name="asl_model")
model.summary()
for layer in model.layers:
    layer.trainable = True


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 230, 230, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9408        conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
scale_conv

res4b28_branch2c (Conv2D)       (None, 14, 14, 1024) 262144      res4b28_branch2b_relu[0][0]      
__________________________________________________________________________________________________
bn4b28_branch2c (BatchNormaliza (None, 14, 14, 1024) 4096        res4b28_branch2c[0][0]           
__________________________________________________________________________________________________
scale4b28_branch2c (Scale)      (None, 14, 14, 1024) 2048        bn4b28_branch2c[0][0]            
__________________________________________________________________________________________________
res4b28 (Add)                   (None, 14, 14, 1024) 0           scale4b28_branch2c[0][0]         
                                                                 res4b27_relu[0][0]               
__________________________________________________________________________________________________
res4b28_relu (Activation)       (None, 14, 14, 1024) 0           res4b28[0][0]                    
__________

In [11]:
sgd = keras.optimizers.SGD(lr = 0.01,momentum=0.0, decay = 1e-4,nesterov = True)

model.compile(optimizer = sgd,
            loss="categorical_crossentropy",
            metrics=["accuracy"])
#checkpointer = ModelCheckpoint(filepath = "/home/t-judai/log_asl201.{epoch:02d}-{val_acc:.2f}.hdf5", monitor = "val_acc", verbose = 1, save_best_only = True, mode = "auto")
history = model.fit_generator(
        train_generator,
        steps_per_epoch=2500,
        epochs=10,
        validation_data = validation_generator,
        validation_steps=1000,
        initial_epoch=8,
        verbose = 1,
        #callbacks = [checkpointer]
        )

Epoch 9/10
 377/2500 [===>..........................] - ETA: 1:16:24 - loss: 0.0021 - acc: 0.9991

KeyboardInterrupt: 

In [ ]:
# list all data in history
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
"""test_generator = test_datagen.flow_from_directory(
        "/home/t-judai/kaggle-link/datasets/grassknoted/asl-alphabet/asl_alphabet_test",
        target_size = (224,224),
        batch_size = 32,
        class_mode = "categorical"
        )
"""
def separateData(data_dir):
    for filename in listdir(data_dir):
        if isfile(join(data_dir, filename)):
            tokens = filename.split('.')
            if tokens[-1] == 'jpg':
                image_path = join(data_dir, filename)
                if not os.path.exists(join(data_dir, tokens[0])):
                    os.makedirs(join(data_dir, tokens[0]))
                copyfile(image_path, join(join(data_dir, tokens[0]), filename))
                os.remove(image_path)
#separateData("/home/t-judai/kaggle-link/datasets/grassknoted/asl-alphabet/asl_alphabet_test")
def plots(ims, figsize=(12,6), rows=4, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
        
#test_imgs, test_labels = next(test_generator)
#plots (test_imgs, titles = test_labels)

#score = model.evaluate_generator(test_generator)
#print (score)

In [ ]:
"""img = "/work/kaggle-link/datasets/grassknoted/asl-alphabet/asl_alphabet_predict/space4.jpg"
file = load_img(img,target_size = (224, 224))
numpy_file = img_to_array(file)
final_file = numpy.expand_dims(numpy_file, axis = 0)
plt.imshow(np.uint8(final_file[0]))
y_prob = model.predict(final_file/255)
y_classes = y_prob.argmax(axis=-1)

print (y_prob)
print (y_classes)"""

In [ ]:
#model.save("/work/7_17_18_resnet152.hdf5")

In [12]:
keras.models.load_model("/work/7_17_18_resnet152.hdf5")

ValueError: Unknown layer: Scale

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    fig_size = plt.rcParams["figure.figsize"]
    fig_size[0] = 10
    fig_size[1]= 12
    plt.rcParams["figure.figsize"] = fig_size

In [ ]:
import itertools
classes = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "del", "nothing", "space"]
pred = np.argmax(res, axis = 1)
matrix = confusion_matrix(validation_generator.classes, pred)
plot_confusion_matrix(matrix, classes, normalize=False)